# Etapa 1: Importação das bibliotecas

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

In [ ]:
!pip install bert-for-tf2

In [ ]:
!pip install sentencepiece

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import tensorflow_hub as hub

In [ ]:
from tensorflow.keras import layers
import bert

# Etapa 2: Carregamento da base de dados

In [ ]:
drive.mount('/content/drive')

In [ ]:
cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ProjetoIA/training.1600000.processed.noemoticon.csv',
                   header = None,
                   names = cols,
                   engine = 'python',
                   encoding='latin-1')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.drop(['id', 'date', 'query', 'user'],
          axis=1,
          inplace=True)

# Etapa 3: Limpeza dos textos

In [ ]:
def limpar_tweet(tweet):
  tweet = BeautifulSoup(tweet, 'lxml').get_text() #Deixar o texto em um formato adequado para fazer a limpeza
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", ' ', tweet)

  return tweet

In [ ]:
teste = data['text'][0]
teste

In [ ]:
result = limpar_tweet(teste)
result

In [ ]:
data_limpo = [limpar_tweet(tweet) for tweet in data.text] #Utiliza a função para limpar todos os tweets

In [ ]:
data_limpo[0:4]

In [ ]:
data_labels = data.sentiment.values #Ajustando os valores do sentimento positivo
data_labels[data_labels == 4] = 1

# Etapa 4: Tokenização

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1', trainable=False) #Pegar modelo
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() #Adicionando o arquivo de vocabulario
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy() #Deixando transformar em minusculo
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
tokenizer.tokenize("My cat is happy.")

In [ ]:
def encode_sentence(sent):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [ ]:
encode_sentence("My dog is hungry.")

In [ ]:
data_entradas = [encode_sentence(sentence) for sentence in data_limpo]

In [ ]:
data_entradas[0]

# Etapa 5: Criação da base de dados

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_entradas)]

In [ ]:
data_with_len[0]

In [ ]:
random.shuffle(data_with_len) #Mistura os dados da base de dados
data_with_len.sort(key=lambda x: x[2]) #Ordenada de forma crescente pegando o tamanho
sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7] #Cria uma nova lista e pega a textos mais relevantes

In [ ]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32)) #Tranformar os dados para que o tensorflow consiga interpretar

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None,), ())) #Definir o numero de registros em cada batch e criar uma lista

In [ ]:
next(iter(all_batched))

In [ ]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE #Quantidade de batches para treinamento
NB_BATCHES

In [ ]:
NB_BATCHES_TEST = NB_BATCHES // 10 #Quantidade de batches para teste
NB_BATCHES_TEST

In [ ]:
all_batched.shuffle(NB_BATCHES) #Embaralhar os dados

In [ ]:
test_dataset = all_batched.take(NB_BATCHES_TEST) #Criar base de dados de teste
train_dataset = all_batched.skip(NB_BATCHES_TEST) #Crirar base de dados de treinamento

# Etapa 6: Criação do modelo

In [ ]:
class DCNN(tf.keras.Model):

  def __init__(self,
               vocab_size,
               emb_dim = 128,
               nb_filters = 50,
               FFN_units=512,
               nb_classes = 2,
               dropout_rate=0.1,
               training = False,
               name="dcnn"):
    super(DCNN, self).__init__(name=name)

    self.embedding = layers.Embedding(vocab_size, emb_dim) #Criando embedding com o tamanho do voabulario x dimensão embedding

    #Definição dos filtros
    self.bigram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 2,
                                padding = 'valid',
                                activation = 'relu')

    self.trigram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 3,
                                padding = 'valid',
                                activation = 'relu')
    self.fourgram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 4,
                                padding = 'valid',
                                activation = 'relu')

    self.pool = layers.GlobalMaxPool1D() #Camada de pooling

    self.dense_1 = layers.Dense(units = FFN_units, activation = 'relu') #Camada mais densa

    self.dropout = layers.Dropout(rate = dropout_rate) #Camada de dropout

    if nb_classes == 2:
      self.last_dense = layers.Dense(units = 1, activation = 'sigmoid')
    else:
      self.leat_dense = layers.Dense(units = nb_classes, activation = 'softmax')

  def call(self, inputs, training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool(x_3)

    merged = tf.concat([x_1, x_2, x_3], axis = -1)
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output




# Etapa 7: Treinamento do modelo

In [ ]:
VOCABULARIO_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256 #Número de neuronios na camada densa
NB_CLASSES = 2
DROPOUT_RATE = 0.2 #Zerar neuronios
NB_EPOCHS = 5

In [ ]:
Dcnn = DCNN(vocab_size=VOCABULARIO_SIZE,
            emb_dim=EMB_DIM,
            nb_filters = NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes = NB_CLASSES,
            dropout_rate = DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
  Dcnn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
else:
  Dcnn.compile(loss = 'sparce_categorical_crossentropy', optimizer = 'adam', metrics = ['sparce_categorical_accuracy'])

In [ ]:
checkpoint_path = "/content/drive/MyDrive/ProjetoIA"

In [ ]:
cktp = tf.train.Checkpoint(Dcnn=Dcnn)

In [ ]:
cktp_manager = tf.train.CheckpointManager(cktp, checkpoint_path, max_to_keep=10)

In [ ]:
if cktp_manager.latest_checkpoint:
  cktp.restore(cktp_manager.latest_checkpoint)
  print('Ultimo checkpoint resturado!')

In [ ]:
class MyCustomCallBack(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    cktp_manager.save()
    print("Checkpoint salvo!".format(checkpoint_path))

In [ ]:
history = Dcnn.fit(train_dataset,
                   epochs = NB_EPOCHS,
                   steps_per_epoch = 100,
                   callbacks=[MyCustomCallBack()]) #Treinamento do modelo